In [1]:
import os
cwd = os.getcwd()
nwd = os.path.dirname(cwd)
os.chdir(nwd)

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
from sklearn.linear_model import LinearRegression
from ghostSHAP.ghost_shap import GhostShap